In [1]:
# Introduction

In [2]:
## Import packages and files

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numpy import exp
from sklearn import preprocessing
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [4]:
heart_disease_training = pd.read_csv("C:/Users/Calvi/Documents/Data Science Toolbox/Assessment 1/data/TrainingDataImputation.csv")
training_y = heart_disease_training.iloc[:, 0]
heart_disease_training_indep = heart_disease_training
del heart_disease_training_indep['HeartDiseaseorAttack']

In [9]:
print("Heart Disease Training Dataset:")
print(heart_disease_training)

Heart Disease Training Dataset:
        HighBP  HighChol  CholCheck   BMI  Smoker  Stroke  Diabetes  \
0          1.0       0.5        1.0  0.27     0.5     0.5  0.333333   
1          1.0       1.0        1.0  0.30     1.0     0.5  0.333333   
2          1.0       1.0        1.0  0.27     1.0     0.5  0.333333   
3          0.5       1.0        0.0  0.31     1.0     0.5  0.333333   
4          1.0       1.0        1.0  0.28     1.0     1.0  0.333333   
...        ...       ...        ...   ...     ...     ...       ...   
412509     1.0       1.0        1.0  0.45     0.5     0.5  0.333333   
412510     1.0       1.0        1.0  0.18     0.5     0.5  0.333333   
412511     0.5       0.5        1.0  0.28     0.5     0.5  0.333333   
412512     1.0       1.0        1.0  0.41     1.0     0.5  0.333333   
412513     1.0       0.5        1.0  0.23     0.5     0.5  0.333333   

        PhysActivity  Fruits  Veggies  ...  AnyHealthcare  NoDocbcCost  \
0                0.0     0.0      0.0  ..

In [10]:
print("Heart Disease Training Classification:")
print(training_y)

Heart Disease Training Classification:
0         1.0
1         1.0
2         1.0
3         1.0
4         1.0
         ... 
412509    0.0
412510    0.0
412511    0.0
412512    0.0
412513    0.0
Name: HeartDiseaseorAttack, Length: 412514, dtype: float64


In [11]:
## PCA

In [12]:
def pcafeatures(data,perc):
    hd_scaled = StandardScaler().fit_transform(data)
    features = hd_scaled.T
    cov_matrix = np.cov(features)
    values, vectors = np.linalg.eig(cov_matrix)
    explained_variances = []
    a = 0
    num = 0
    for i in range(len(values)):
        explained_variances.append(values[i] / np.sum(values))
    for i in range(len(explained_variances)):
        while a < perc:
            a += explained_variances[i]*100
            num += 1
    pc_feat = ['PC{}'.format(i+1) for i in range(num)]
    return pc_feat,num

In [13]:
class PCA:
    
    def __init__(self,X):
        self.n_components = pcafeatures(X, 95)[1]
        self.components = None
        self.mean = None
        
    def fit(self,X):
        # mean
        self.mean = np.mean(X, axis=0)
        X = X-self.mean
        # covariance
        cov = np.cov(X.T)
        # eigenvectors, eigenvalues
        eigenvalues, eigenvectors = np.linalg.eig(cov)
        # v[:, i]
        # sort eigenvectors
        eigenvectors = eigenvectors.T
        idxs = np.argsort(eigenvalues)[::-1]
        eigenvalues = eigenvalues[idxs]
        eigenvectors = eigenvectors[idxs]
        
        # store first n eigenvectors
        self.components = eigenvectors[0:self.n_components]
        
    def transform(self,X):
        X = X - self.mean
        return np.dot(X, self.components.T)

In [14]:
pca = PCA(heart_disease_training_indep)
pca.fit(heart_disease_training_indep)
hda_pca_tr = pca.transform(heart_disease_training_indep)

In [15]:
print("PCA of Heart Disease Training Dataset:")
print(hda_pca_tr)

PCA of Heart Disease Training Dataset:
[[ 1.23049933  0.78767842 -0.37535516 ... -0.34321415  0.2977642
  -0.14318878]
 [-0.15674317  0.6804508  -0.92494614 ...  0.34558831 -0.08096224
   0.20903433]
 [-0.09306808 -0.50254292 -0.66936754 ... -0.26274617  0.30885442
   0.18105975]
 ...
 [-0.04701631  0.39857191  0.42281489 ...  0.1399973  -0.14284745
   0.30420101]
 [ 0.98766112  0.63285981 -0.65643159 ... -0.46548152 -0.17183657
   0.10120914]
 [ 0.15395439 -0.43956448 -0.13403093 ...  0.21435531  0.29552384
   0.08346948]]


In [16]:
# Logistic Regression

In [17]:
## Preprocessing

In [18]:
scaler = preprocessing.StandardScaler().fit(hda_pca_tr)
hda_pca_tr = scaler.transform(hda_pca_tr)

In [19]:
print(hda_pca_tr.mean(axis=0))

[-1.18037732e-16 -1.58978541e-17  1.12910667e-15  2.08022089e-16
 -1.86653242e-16 -7.39108922e-17 -6.46958424e-16  4.37930036e-16]


In [ ]:
scaler = StandardScaler()
hda_pca_tr = scaler.fit_transform(hda_pca_tr)
hda_pca_tr1 = scaler.transform(hda_pca_tr)

In [ ]:
def sigmoid(z):
    return 1/(1 + exp(-z))

def initialise_weights(dim):
  # numpy.zeros_like function return an array of zeros with the same shape and type as a given array
    w = np.zeros_like(dim)
    b = 0
    return w, b

def logloss(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    log_loss = -1 * np.mean( y_true*np.log10(y_pred) + (1-y_true)*np.log10(1-y_pred))
    return log_loss


def gradient_dw(x, y, w, b, alpha, N): 
    dw = x * (y-sigmoid(np.dot(w.T,x)+b)) - ((alpha*w*w)/N)
    return dw

def gradient_db(x, y, w, b):
    db = y-sigmoid(np.dot(w.T,x)+b)
    return db 

In [ ]:
x = np.linspace(-20,20)
y = sigmoid(x)
plt.plot(x,sigmoid(x))
plt.title("Sigmoid Function")
plt.show()

In [ ]:
def train(X_train, y_train, X_test, y_test, epochs, alpha, eta0):
    w, b = initialise_weights(X_train[0])
    N = len(X_train)
    log_loss_train = []
    log_loss_test = []
    
    for i in range(0, epochs):
        
        for j in range(N):
            grad_dw = gradient_dw(X_train[j], y_train[j], w, b, alpha, N)
            grad_db = gradient_db(X_train[j], y_train[j], w, b)
            w = np.array(w) + (eta0 * np.array(grad_dw))
            b = b + (eta0 * grad_db)

    # predict the output of x_train[for all data points in X_train] using w and b
        predict_train = []
        for m in range(len(y_train)):
            z = np.dot(w, X_train[m])+b
            predict_train.append(sigmoid(z)) 
    
    # store all the train loss values in a list
        train_loss = logloss(y_train, predict_train)

    # predict the output of x_test[for all data points in X_test] using w,b
        predict_test = []
        for m in range(len(y_test)):
            z = np.dot(w, X_test[m])+b
            predict_test.append(sigmoid(z))
    
    # store all the test loss values in a list
        test_loss = logloss(y_test, predict_test)

    # we can also compare previous loss and current loss, 
    #if loss is not updating then stop the process and return w,b
        if log_loss_train and train_loss > log_loss_train[-1]: 
            return w, b, log_loss_train, log_loss_test 
    
        log_loss_train.append(train_loss)
        log_loss_test.append(test_loss)

    return w, b, log_loss_train, log_loss_test

In [ ]:
alpha  = 0.0001
eta0   = 0.0001
epochs = 100

In [ ]:
w, b, log_loss_train, log_loss_test = train(hda_pca_tr, training_y, hda_pca_tr1, training_y, epochs, alpha, eta0)

print("weight coefficient is:", w)
print("y-intercept is:", b)

In [ ]:
## Prediction formula

In [ ]:
def predict(X):
    z = np.dot(w,X.T)+b
    list = []
    for i in sigmoid(z):
            if i>0.5:
                list.append(1)
            else:
                list.append(0)
    return list

In [ ]:
predict_train_y = predict(hda_pca_tr)

In [ ]:
## Test

In [ ]:
heart_disease_test = pd.read_csv("https://raw.githubusercontent.com/erinp0/DST-Assessment-1/main/Erin%20Pollard/test_imputed.csv")
test_y = heart_disease_test.iloc[:, 0]
heart_disease_test_indep = heart_disease_test
del heart_disease_test_indep['HeartDiseaseorAttack']

hda_pca_test= PCA(heart_disease_test_indep)
pca.fit(heart_disease_test_indep)
hda_pca_test = pca.transform(heart_disease_test_indep)
hda_pca_tr = scaler.transform(hda_pca_tr)

predict_test_y = predict(hda_pca_test)

In [ ]:
## Accuracy

In [ ]:
def accuracy(y,y_hat):
    accuracy = 0
    for i in range(len(y)):
        if y[i] == y_hat[i]:
            accuracy += 1
        else:
            continue
    accuracy = accuracy/len(y)
    return accuracy

In [ ]:
print("training accuracy:", accuracy(training_y,predict_train_y))
print("test accuracy:", accuracy(test_y,predict_test_y))

In [ ]:
x1 = np.linspace(0,50)
y = train(hda_pca_tr, training_y, hda_pca_test, predict_test_y, int(x1), alpha, eta0)
plt.plot(x1,y[2], label = "Training loss")
plt.plot(x1,y[3], label = "Training loss")
plt.xlabel("Epoch")
plt.ylabel("Log loss")
plt.legend()
plt.title("Log loss vs training iterations")
plt.show()